### Packages

In [1]:
import sys
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.models import load_model
from keras_metrics import KerasMetrics
from keras.callbacks import EarlyStopping
from keras import optimizers
import numpy as np
import random
import pickle
import decoding2
import decoding_function as df
from Files import nodes
import os
import utility as ut
import livelossplot
sys.path.append('../')

Using TensorFlow backend.


In [4]:
def pipeline(verbs):
    """
    Pipeline receives a list of verbs and predicts a conjugated form for them.
    The prediction is saved in an output file.

    :verbs type: list
    :r type: string
    """
    import coding_function as cf
    from time import gmtime, strftime
    import itertools
    time = strftime("%Y-%m-%d_%H:%M:%S", gmtime())
    name = "output_" + time + ".txt"

    correct = {'#pega#': '#pEgu#', '#sega#': '#sEgu#', '#seka#': '#sEku#',
               '#leva#': '#lEvu#', '#ora#': '#Oru#', '#mora#': '#mOru#',
               '#posta#': '#pOtu#', '#joga#': '#jOgu#', '#sortia#': '#soiu#',
               '#media#': '#meiu#', '#kompo#': '#koiu#', '#po#': '#poiu#', 
               '#menti#': '#mitu#', '#tosi#': '#tusu#', '#kobri#': 
               '#kuro#', '#faze#': '#fasu#', '#mata#': '#matu#', '#paga#': 
               '#pagu#', '#sai#': '#saiu#', '#bate#': '#batu#', '#kome#': 
               '#komu#','#nota#': '#nOta#','#soka#': '#sOku#','#loga#': '#lOga#',
               '#limpa#': '#lipu#','#kasa#': '#kasu#'}

#     f = open('Files/Results/'+ name, "w")
#     f.write("train data set:" + path + '\n' + 'Results:\n' + 'model name: ' + load + '\n')
    test_list = []
    for i in verbs:
        coding = cf.coding(i)
        test_list.append(coding)
    test_list = np.array(test_list)
    prediction = model.predict(test_list)
    sum = 0
    for i,j in list(zip(verbs, prediction)):
#         f.write('verb: '+ i + ", expected: " + correct[i] + ", prediction: " + 
#                 decoding2.decoding(j) + '\n')
        decoded = decoding2.decoding(j)
        try: 
            print('input: '+ i + ", expected: " + correct[i] + ", output: " + 
                  decoded + '\n')
            if decoded == correct[i]:
                sum = sum + 1 
        except: 
            print('input: '+ i + " -> output: " + decoded + '\n')
        
   
    print("Accuracy (verbs): %.2f%%" % (sum/len(verbs)*100 ))
    scores = model.evaluate(X, Y)
#     f.write("\n%s: %.2f%% \n%s: %.2f%% \n%s: %.2f%%" % (model.metrics_names[0], scores[1]*100,
#                                                         model.metrics_names[1], scores[1]*100, 
#                                                         model.metrics_names[2], scores[2]*100,
#                                                         model.metrics_names[3], scores[3]*100 ))
    print("\n%s: %.2f%% \n%s: %.2f%% \n%s: %.2f%% \n%s: %.2f%%" % (model.metrics_names[0], scores[0]*100,
                                                                   model.metrics_names[1], scores[1]*100, 
                                                                   model.metrics_names[2], scores[2]*100,
                                                                   model.metrics_names[3], scores[3]*100 ))
    
    
    #f.close()
    return #print("\nPredictions saved in the file: " + name) 

### 1. Load Data

In [21]:
#path = '../data/prop_55.csv' 
#path = '../data/prop_65.csv'
#path = '../data/prop_75.csv'
path = '../data/prop_85.csv'
#path = '../data/prop_95.csv'
#path = 'Files/50_50_morar.csv'
X,Y = ut.load_data(path)
batch = len(X)
batch

390

#### 1.2 Verify length of unique verbs

In [10]:
path = '../data/prop_65.csv'
X,Y = ut.load_data(path, verbose = True)

unique verbs: 340 

lenght of data set: 565


### 2. Fit Neural Net (Optional)

In [22]:
# 1. Define Model
model = Sequential()
model.add(Dense(460, input_shape=(460,), activation='sigmoid'))


# 2. Compile model
keras_metrics = KerasMetrics()
model.compile( 
    optimizer='adam', 
    loss='mean_squared_error',
    metrics = [ keras_metrics.fbeta_score,
               keras_metrics.recall,
               keras_metrics.precision]
            )

# 3. Fit model
#stopper = EarlyStopping(monitor='loss', min_delta=0.00005, patience=50, verbose=1, mode='max')
model.fit(X,Y,epochs = 400, batch_size=batch,verbose=False)#,callbacks= [stopper])

# 4. Evaluate model
scores = model.evaluate(X, Y)
print("\n%s: %.2f%% \n%s: %.2f%% \n%s: %.2f%% \n%s: %.2f%%" % (model.metrics_names[0], scores[0]*100,
                                                               model.metrics_names[1], scores[1]*100, 
                                                               model.metrics_names[2], scores[2]*100,
                                                               model.metrics_names[3], scores[3]*100 ))

# 5. Save model
model.save('../data/prop_85')

 32/390 [=>............................] - ETA: 1s
loss: 1.42% 
fbeta_score: 99.14% 
recall: 98.80% 
precision: 99.49%


### 2. (Or Load Trained Model)
For example: 

In [5]:
keras_metrics = KerasMetrics()
model = load_model('../data/prop_75',custom_objects={'fbeta_score': keras_metrics.fbeta_score,
                                                     'recall': keras_metrics.recall,
                                                     'precision': keras_metrics.precision})

#### 2.1 Train existing model

model.fit(X,Y,epochs = 150, batch_size=batch,verbose=False,callbacks= [stopper])

### 3. Check Results
Test the prediction for some verbs.

pipeline(['#pega#','#sega#','#seka#','#leva#','#ora#','#mora#', '#posta#',
          '#joga#','#sortia#','#media#','#kompo#','#po#','#tendi#','#jenti#',
          '#menti#','#hendi#','#tosi#','#kobri#','#faze#','#mata#','#paga#',
          '#sai#','#bate#','#kome#'])
    

|       	|       	| front 	| middle  	| back    	|
|:-----:	|-------	|------:	|---------	|---------	|
|       	|       	| v/u   	| v/u     	| v/u     	|
|  **int**  | **stop**  | b/p   	| d/t     	| g/k     	|
|       	| **nasal** | m/-   	| n/-     	| -       	|
| **cont**  | **fric**  | v/f   	| z/s    	| j/x   	|
|       	| **liq**   | l/-   	| r/-     	| -/h     	|
| **vowel** | **high**  | e/i   	| -       	| o/u     	|
|       	| **low**   | -/E   	| -/a     	| -/O     	|

## Results:

## Proporcao 55% Irregulares

|     Model Name     | Test File     | Total Irregular Verbs | Total Verbs | Proportion | Epochs | Batch Size  |
|:------------------:|---------------|----------------------:|-------------|------------|--------|-------------|
| prop_55    | prop_55       |   209    |  464     |    55%     | 400    |  464         |

In [17]:
path = '../data/prop_55.csv'
X,Y = ut.load_data(path, verbose = True)
keras_metrics = KerasMetrics()
load = '../data/prop_55'
model = load_model(load,custom_objects={'fbeta_score': keras_metrics.fbeta_score,
                                                     'recall': keras_metrics.recall,
                                                     'precision': keras_metrics.precision})
pipeline(['#pega#','#sega#','#seka#','#leva#','#ora#','#mora#', '#posta#',
          '#joga#','#sortia#','#media#','#kompo#','#po#','#menti#','#tosi#',
          '#kobri#','#faze#','#mata#','#paga#',
          '#sai#','#bate#','#kome#'])

unique verbs: 341 

lenght of data set: 452
input: #pega#, expected: #pEgu#, output: #peju#

input: #sega#, expected: #sEgu#, output: #segu#

input: #seka#, expected: #sEku#, output: #seku#

input: #leva#, expected: #lEvu#, output: #levu#

input: #ora#, expected: #Oru#, output: #orru#

input: #mora#, expected: #mOru#, output: #moru#

input: #posta#, expected: #pOtu#, output: #pOtu#

input: #joga#, expected: #jOgu#, output: #jogu#

input: #sortia#, expected: #soiu#, output: #soiu#

input: #media#, expected: #meiu#, output: #meiu#

input: #kompo#, expected: #koiu#, output: #koiu#

input: #po#, expected: #poiu#, output: #poiu#

input: #menti#, expected: #mitu#, output: #mitu#

input: #tosi#, expected: #tusu#, output: #tufu#

input: #kobri#, expected: #kuro#, output: #koru#

input: #faze#, expected: #fasu#, output: #fasu#

input: #mata#, expected: #matu#, output: #matu#

input: #paga#, expected: #pagu#, output: #pagu#

input: #sai#, expected: #saiu#, output: #saiu#

input: #bate#, expected

## Proporcao 65% Irregulares


|     Model Name     | Test File     | Total Irregular Verbs | Total Verbs | Proportion | Epochs | Batch Size  | Total Unique Verbs
|:------------------:|---------------|------------:|-------------|------------|--------|-------------|-------------|
| prop_65     | prop_65       |      209        |  565        |    65%     | 400    |  565         |340|

In [11]:
path = '../data/prop_65.csv'
X,Y = ut.load_data(path)
keras_metrics = KerasMetrics()
load = '../data/testing'
model = load_model(load,custom_objects={'fbeta_score': keras_metrics.fbeta_score,
                                                     'recall': keras_metrics.recall,
                                                     'precision': keras_metrics.precision})
pipeline(['#pega#','#sega#','#seka#','#leva#','#ora#','#mora#', '#posta#',
          '#joga#','#sortia#','#media#','#kompo#','#po#','#menti#','#tosi#',
          '#kobri#','#faze#','#mata#','#paga#',
          '#sai#','#bate#','#kome#'])

input: #pega#, expected: #pEgu#, output: #peju#

input: #sega#, expected: #sEgu#, output: #segu#

input: #seka#, expected: #sEku#, output: #seku#

input: #leva#, expected: #lEvu#, output: #levu#

input: #ora#, expected: #Oru#, output: #orru#

input: #mora#, expected: #mOru#, output: #moru#

input: #posta#, expected: #pOtu#, output: #bOtu#

input: #joga#, expected: #jOgu#, output: #jogu#

input: #sortia#, expected: #soiu#, output: #suiu#

input: #media#, expected: #meiu#, output: #meiu#

input: #kompo#, expected: #koiu#, output: #koiu#

input: #po#, expected: #poiu#, output: #poiu#

input: #menti#, expected: #mitu#, output: #mitu#

input: #tosi#, expected: #tusu#, output: #tufu#

input: #kobri#, expected: #kuro#, output: #koru#

input: #faze#, expected: #fasu#, output: #fasu#

input: #mata#, expected: #matu#, output: #matu#

input: #paga#, expected: #pagu#, output: #pagu#

input: #sai#, expected: #saiu#, output: #saiu#

input: #bate#, expected: #batu#, output: #baiu#

input: #kome#, exp

## Proporcao 75% Irregulares

|     Model Name     | Test File     | Total Irregular Verbs | Total Verbs | Proportion | Epochs | Batch Size  |Total Unique Verbs|
|:------------------:|---------------|----------------------:|-------------|------------|--------|-------------|----|
| prop_75       | prop_75       |            300          |  388        |    75%     | 400   |  388         | 244|

In [14]:
path = '../data/prop_75.csv'
X,Y = ut.load_data(path, verbose = True)
keras_metrics = KerasMetrics()
load = '../data/prop_75'
model = load_model(load, custom_objects={'fbeta_score': keras_metrics.fbeta_score,
                                                     'recall': keras_metrics.recall,
                                                     'precision': keras_metrics.precision})
pipeline(['#pega#','#sega#','#seka#','#leva#','#ora#','#mora#', '#posta#',
          '#joga#','#sortia#','#media#','#kompo#','#po#','#menti#','#tosi#',
          '#kobri#','#faze#','#mata#','#paga#',
          '#sai#','#bate#','#kome#'])

unique verbs: 244 

lenght of data set: 388
input: #pega#, expected: #pEgu#, output: #peku#

input: #sega#, expected: #sEgu#, output: #segu#

input: #seka#, expected: #sEku#, output: #siku#

input: #leva#, expected: #lEvu#, output: #lezu#

input: #ora#, expected: #Oru#, output: #orru#

input: #mora#, expected: #mOru#, output: #muru#

input: #posta#, expected: #pOtu#, output: #pOtu#

input: #joga#, expected: #jOgu#, output: #jogu#

input: #sortia#, expected: #soiu#, output: #suiu#

input: #media#, expected: #meiu#, output: #meiu#

input: #kompo#, expected: #koiu#, output: #koiu#

input: #po#, expected: #poiu#, output: #poiu#

input: #menti#, expected: #mitu#, output: #mitu#

input: #tosi#, expected: #tusu#, output: #tufu#

input: #kobri#, expected: #kuro#, output: #koru#

input: #faze#, expected: #fasu#, output: #fasu#

input: #mata#, expected: #matu#, output: #matu#

input: #paga#, expected: #pagu#, output: #bagu#

input: #sai#, expected: #saiu#, output: #saiu#

input: #bate#, expected

## Proporcao 85% Irregulares

|     Model Name     | Test File     | Total Irregular Verbs | Total Verbs | Proportion | Epochs | Batch Size  | Total Unique Verbs|
|:------------------:|---------------|----------------------:|-------------|------------|--------|-------------|---|
| prop_85       | prop_85       |             340         |  390        |    85%     | 400   |  390         | 212|

In [23]:
path = '../data/prop_85.csv'
X,Y = ut.load_data(path, verbose = True)
keras_metrics = KerasMetrics()
load = '../data/prop_85'
model = load_model(load,custom_objects={'fbeta_score': keras_metrics.fbeta_score,
                                                     'recall': keras_metrics.recall,
                                                     'precision': keras_metrics.precision})
pipeline(['#pega#','#sega#','#seka#','#leva#','#ora#','#mora#', '#posta#',
          '#joga#','#sortia#','#media#','#kompo#','#po#','#menti#','#tosi#',
          '#kobri#','#faze#','#mata#','#paga#',
          '#sai#','#bate#','#kome#'])

unique verbs: 212 

lenght of data set: 390
input: #pega#, expected: #pEgu#, output: #peku#

input: #sega#, expected: #sEgu#, output: #sigu#

input: #seka#, expected: #sEku#, output: #siku#

input: #leva#, expected: #lEvu#, output: #levu#

input: #ora#, expected: #Oru#, output: #orru#

input: #mora#, expected: #mOru#, output: #muru#

input: #posta#, expected: #pOtu#, output: #pOtu#

input: #joga#, expected: #jOgu#, output: #jogu#

input: #sortia#, expected: #soiu#, output: #suiu#

input: #media#, expected: #meiu#, output: #meiu#

input: #kompo#, expected: #koiu#, output: #koiu#

input: #po#, expected: #poiu#, output: #poiu#

input: #menti#, expected: #mitu#, output: #mitu#

input: #tosi#, expected: #tusu#, output: #tupu#

input: #kobri#, expected: #kuro#, output: #koru#

input: #faze#, expected: #fasu#, output: #fasu#

input: #mata#, expected: #matu#, output: #matu#

input: #paga#, expected: #pagu#, output: #pagu#

input: #sai#, expected: #saiu#, output: #saiu#

input: #bate#, expected

## Proporcao 95% Irregulares

|     Model Name     | Test File     | Total Irregular Verbs | Total Verbs | Proportion | Epochs | Batch Size  | Total Unique Verbs|
|:------------------:|---------------|----------------------:|-------------|------------|--------|-------------|---|
| prop_95       | prop_95       |             380         |  395        |    95%     | 400   |  395         | 181|

In [20]:
path = '../data/prop_95.csv'
X,Y = ut.load_data(path, verbose = True)
keras_metrics = KerasMetrics()
load = '../data/prop_95'
model = load_model(load, custom_objects={'fbeta_score': keras_metrics.fbeta_score,
                                                     'recall': keras_metrics.recall,
                                                     'precision': keras_metrics.precision})
pipeline(['#pega#','#sega#','#seka#','#leva#','#ora#','#mora#', '#posta#',
          '#joga#','#sortia#','#media#','#kompo#','#po#','#menti#','#tosi#',
          '#kobri#','#faze#','#mata#','#paga#',
          '#sai#','#bate#','#kome#'])

unique verbs: 181 

lenght of data set: 395
input: #pega#, expected: #pEgu#, output: #pEku#

input: #sega#, expected: #sEgu#, output: #sigu#

input: #seka#, expected: #sEku#, output: #siku#

input: #leva#, expected: #lEvu#, output: #levu#

input: #ora#, expected: #Oru#, output: #earu#

input: #mora#, expected: #mOru#, output: #mEru#

input: #posta#, expected: #pOtu#, output: #pOtu#

input: #joga#, expected: #jOgu#, output: #xogu#

input: #sortia#, expected: #soiu#, output: #soiu#

input: #media#, expected: #meiu#, output: #meiu#

input: #kompo#, expected: #koiu#, output: #koiu#

input: #po#, expected: #poiu#, output: #poiu#

input: #menti#, expected: #mitu#, output: #mitu#

input: #tosi#, expected: #tusu#, output: #tutu#

input: #kobri#, expected: #kuro#, output: #koru#

input: #faze#, expected: #fasu#, output: #fasu#

input: #mata#, expected: #matu#, output: #matu#

input: #paga#, expected: #pagu#, output: #pagu#

input: #sai#, expected: #saiu#, output: #saiu#

input: #bate#, expected

In [54]:
keras_metrics = KerasMetrics()
load = '../data/prop_95'
model = load_model(load, custom_objects={'fbeta_score': keras_metrics.fbeta_score,
                                                     'recall': keras_metrics.recall,
                                                     'precision': keras_metrics.precision})
pipeline(['#tega#'])

input: #tega# -> output: #tigu#

Accuracy (verbs): 0.00%
 32/192 [====>.........................] - ETA: 1s
loss: 4.74% 
fbeta_score: 84.77% 
recall: 83.61% 
precision: 85.99%


### Experimento 50 - 50

In [49]:
model = load_model('../data/test_5050',custom_objects={'fbeta_score': keras_metrics.fbeta_score,
                                                     'recall': keras_metrics.recall,
                                                     'precision': keras_metrics.precision})
pipeline(['#nota#','#soka#','#loga#','#bate#','#limpa#','#kasa#'])

input: #nota#, expected: #nOta#, output: #dOtu#

input: #soka#, expected: #sOku#, output: #sOku#

input: #loga#, expected: #lOga#, output: #logu#

input: #bate#, expected: #batu#, output: #batu#

input: #limpa#, expected: #lipu#, output: #lipu#

input: #kasa#, expected: #kasu#, output: #kasu#

Accuracy (verbs): 66.67%
 32/192 [====>.........................] - ETA: 1s
loss: 0.73% 
fbeta_score: 99.87% 
recall: 99.75% 
precision: 99.99%


In [1]:
import livelossplot

Using TensorFlow backend.


In [9]:
import coding_function as cf

In [10]:
verbs = ['#posta#']
test_list = []
for i in verbs:
    coding = cf.coding(i)
    test_list.append(coding)
test_list = np.array(test_list)
prediction = model.predict(test_list)

In [11]:
prediction

array([[ 0.01885734,  0.01925963,  0.20835543,  0.06943061,  0.20616981,
         0.0709679 ,  0.04838099,  0.22368896,  0.16970563,  0.12240665,
         0.0206362 ,  0.01205894,  0.91308624,  0.24333856,  0.68530637,
         0.1774959 ,  0.01764845,  0.72755343,  0.33703929,  0.73652261,
         0.03283259,  0.06553374,  0.04157919,  0.05008816,  0.0994515 ,
         0.05351605,  0.06052314,  0.04473304,  0.07151185,  0.05618191,
         0.02789558,  0.01436698,  0.02050743,  0.01129203,  0.06942753,
         0.01098618,  0.05711168,  0.09250909,  0.00476367,  0.10850174,
         0.04378862,  0.02906369,  0.28343081,  0.04388338,  0.26610306,
         0.1870368 ,  0.0226121 ,  0.11979822,  0.15712078,  0.14190321,
         0.87817395,  0.04864824,  0.05664408,  0.10420375,  0.90287876,
         0.13406143,  0.92652106,  0.04734195,  0.916224  ,  0.1086485 ,
         0.02241066,  0.90523285,  0.02660097,  0.09925672,  0.82048297,
         0.03158227,  0.83003283,  0.0138406 ,  0.8